In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
data = pd.read_csv('checkins.dat', sep = '|')

C:\Users\ekaterina.adischeva\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.columns = data.columns.str.strip() 

In [4]:
data = data.iloc[1:]
data = data.iloc[:-1]

In [5]:
for columns in ['latitude', 'longitude']:
    data[columns] = data[columns].str.strip() 

In [6]:
data = data[data['latitude'] != '']

In [7]:
data = data.head(100000)

In [8]:
data = data[['latitude', 'longitude']]

In [9]:
from sklearn.cluster import MeanShift, estimate_bandwidth

In [10]:
bandwidth=0.1

ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(data)

labels = ms.labels_
cluster_centers = ms.cluster_centers_

In [11]:
data['labels'] = labels

In [12]:
data['labels_qnt'] = data.groupby(['labels'])['labels'].transform('sum')

In [13]:
data = data[data['labels_qnt'] >= 15]

In [14]:
office_locations = [[33.751277, -118.188740,  'Los Angeles'],
[25.867736, -80.324116, 'Miami'],
[51.503016, -0.075479,  'London'],
[52.378894, 4.885084, 'Amsterdam'],
[39.366487, 117.036146, 'Beijing'],
[-33.868457, 151.205134, 'Sydney']] 
office_locations_df = pd.DataFrame(office_locations, columns = ['latitude', 'longitude', 'city'])  

In [20]:
office_locations_df

,latitude,longitude,city
0,33.751277,-118.188740,Los Angeles
1,25.867736,-80.324116,Miami
2,51.503016,-0.075479,London
3,52.378894,4.885084,Amsterdam
4,39.366487,117.036146,Beijing
5,-33.868457,151.205134,Sydney


In [21]:
cluster_centers_df =  pd.DataFrame(cluster_centers, columns = ['latitude', 'longitude'])  

In [27]:
cluster_centers_dist = pd.DataFrame()
for index, row in office_locations_df.iterrows():
    cluster_centers_dist_ = cluster_centers_df.copy()
    cluster_centers_dist_['city'] = row['city']
    cluster_centers_dist_['dist'] = ((cluster_centers_dist_['latitude'] - row['latitude'])**2 +
                                    (cluster_centers_dist_['longitude'] - row['longitude'])**2)**(1/2)
    
    cluster_centers_dist = cluster_centers_dist.append(cluster_centers_dist_)

In [51]:
ans = cluster_centers_dist.sort_values('dist').head(1)[['latitude', 'longitude']].values

In [57]:
str_ans = np.array2string(ans).replace('[','').replace(']','')

In [60]:
str_ans

'-33.86063043 151.20477593'

In [58]:
def write_answer_2(ans):
    with open("fsq_answer.txt", "w") as fout:
        fout.write(ans)

In [61]:
write_answer_2(str_ans)